# 단어 임베딩
## Embedding 층을 사용해 단어 임베딩 학습하기
* 단어 벡터 사이에 조금 더 추상적이고 기하학적인 관계를 얻기 위해 단어 사이에 있는 의미 관계를 반영하기 
    * 언어를 기하학적 공간에 매핑하기 
    * 잘 구축된 임베딩 공간에서는 동의어가 비슷한 단어 벡터로 임베딩될 것.. 
    * 일반적으로 두 단어 벡터 사이의 거리(L2 거리)는 이 단어 사이의 의미 거리와 관계(서로 의미가 다른 단어는 멀리 떨어진 위치에, 비슷한 단어들은 가까이)

### ```Embedding```
* 크기가 (samples, sequence_length)인 2D 정수 텐서를 입력받음
* 크기가 (samples, sequence_length, embedding_dimensionality)인 3D 실수형 텐서를 반환

### Embedding 층의 객체 생성하기

In [1]:
from keras.layers import Embedding

# 가능한 토큰의 개수(1,000: 단어 인덱스 최댓값 + 1), 임베딩 차원(64)
embedding_layer = Embedding(1000, 64)

### Embedding층에 사용할 IMDB 데이터 로드

In [2]:
from keras.datasets import imdb
from keras import preprocessing

# 특성으로 사용할 단어의 수
max_features = 10000
# 사용할 텍스트의 길이(가장 자주 사용되는 max_features 개의 단어만 사용)
maxlen = 20

# 정수 리스트로 데이터 로드
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 리스트를 (samples, maxlen) 크기의 2D 정수 텐서로 변환
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

### IMDB 데이터에 Embedding층과 분류기 사용하기

In [3]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
# 나중에 임베딩된 입력을 Flatten 층에서 펼치기 위해 Embedding 층에 input_length를 지정
model.add(Embedding(10000, 8, input_length=maxlen))
# Embedding 층의 출력 크기는 (samples, maxlen, 8)가 된다!

# 3D 임베딩 텐서를 (samples, maxlen * 8) 크기의 2D 텐서로 펼치기
model.add(Flatten())

# 분류기 추가
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 8)             80000     
                                                                 
 flatten (Flatten)           (None, 160)               0         
                                                                 
 dense (Dense)               (None, 1)                 161       
                                                                 
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


2022-03-28 17:17:10.796218: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-28 17:17:10.796516: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-28 17:17:10.908050: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-28 17:17:11.116960: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


621/625 [============================>.] - ETA: 0s - loss: 0.6737 - acc: 0.6116

2022-03-28 17:17:16.421345: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 6s 9ms/step - loss: 0.6736 - acc: 0.6119 - val_loss: 0.6287 - val_acc: 0.6932
Epoch 2/10
625/625 [==============================] - 5s 8ms/step - loss: 0.5499 - acc: 0.7490 - val_loss: 0.5303 - val_acc: 0.7282
Epoch 3/10
625/625 [==============================] - 5s 8ms/step - loss: 0.4653 - acc: 0.7847 - val_loss: 0.5017 - val_acc: 0.7446
Epoch 4/10
625/625 [==============================] - 5s 8ms/step - loss: 0.4239 - acc: 0.8055 - val_loss: 0.4931 - val_acc: 0.7534
Epoch 5/10
625/625 [==============================] - 5s 8ms/step - loss: 0.3963 - acc: 0.8231 - val_loss: 0.4925 - val_acc: 0.7556
Epoch 6/10
625/625 [==============================] - 5s 8ms/step - loss: 0.3733 - acc: 0.8355 - val_loss: 0.4974 - val_acc: 0.7534
Epoch 7/10
625/625 [==============================] - 5s 8ms/step - loss: 0.3535 - acc: 0.8487 - val_loss: 0.5021 - val_acc: 0.7524
Epoch 8/10
625/625 [==============================] - 5s 8ms/step - loss: 0.3345 - acc:

In [8]:
import numpy as np
print(np.mean(history.history['val_acc']))

0.7429000318050385


---
## 사전 훈련된 단어 임베딩 사용하기

> 풀려는 문제와 함께 단어 임베딩을 학습하는 대신에 미리 계산된 임베딩 공간에서 임베딩 벡터를 로드하기 
* 뛰어난 구조와 유용한 성질을 가진 임베딩 공간 -> 언어 구조의 일반적인 측면을 잡아낼 수 있음! 
* 자연어 처리에서 사전 훈련된 단어 임베딩을 사용하는 이유?: 충분한 데이터가 없어서 자신만의 좋은 특성을 학습하지 못하지만 꽤 일반적인 특성이 필요할 때..

### 토큰화 되지 않은 IMDB 원본 데이터 받기

In [11]:
import os

imdb_dir = '../datasets/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding='utf8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

### 데이터 토큰화

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # 100개 단어 이후는 버리기
training_samples = 200  # 훈련 샘플: 200개
validation_samples = 10000  # 검증 샘플: 10,000개
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('데이터 텐서의 크기:', data.shape)
print('레이블 텐서의 크기:', labels.shape)

# 데이터를 훈련 세트와 검증 세트로 분할
# 샘플이 순서대로 있기 때문에 (부정 샘플이 모두 나온 후에 긍정 샘플 나옴) 먼저 데이터를 섞기
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

88582개의 고유한 토큰을 찾았습니다.
데이터 텐서의 크기: (25000, 100)
레이블 텐서의 크기: (25000,)


### GloVe 단어 임베딩 전처리
.txt파일을 파싱하여 단어에 상응하는 벡터 표현을 매핑하는 인덱스 만들기

In [14]:
glove_dir = '../datasets/glove-6b'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('%s개의 단어 벡터를 찾았습니다.' % len(embeddings_index))

400000개의 단어 벡터를 찾았습니다.


```Embedding``` 층에 주입할 수 있도록 임베딩 행렬을 만들기  
행렬의 크기: (max_words, embedding_dim)  
행렬의 i번째 원소: (토큰화로 만든) 단어 인덱스의 i번째 단어에 상응하는 embedding_dim 차원 벡터  
인덱스 0: 어떤 단어나 토큰도 아닐 경우

In [15]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # 임베딩 인덱스에 없는 단어는 모두 0으로
            embedding_matrix[i] = embedding_vector